In [1]:
import pandas as pd

In [2]:
file1 = open('demo_battle.txt', 'r')
battle_log = file1.readlines()

In [3]:
def move_name_cleaner(move):
    move = move.replace(" ", "")
    move = move.replace("_", " ")
    return(move.title())

def process_attack(df):
    attacker = df.iloc[0].value.split(' used')[0]
    if attacker.find('team 1') != -1:
        attacker = 1
        defender = 2
    else:
        attacker = 2
        defender = 1
    attack_name = move_name_cleaner(df.iloc[0].value.split(' used')[1].split('on')[0])
    if (df.category.iloc[0] == "'Attack'"):
        charged_string = "|-prepare|p" + str(attacker) + "a: " + current_battlers[attacker] + "|Geomancy"
        battle_render.append(charged_string)
        if (df['category'].str.contains("'Shield'").max()):
            shield_string = "|-singleturn|p" + str(defender) + "a: "+ current_battlers[defender] +"|Protect"
            battle_render.append(shield_string)
    move_string = "|move|p"+ str(attacker) + "a: " + current_battlers[attacker] + "|" + attack_name + "|p" + str(defender) + "a: " + current_battlers[defender]
    battle_render.append(move_string)
    move_damage_index = df[df.category.isin(["'Charged Move Damage'", "'Fast Attack Damage'"])].index
    current_hp = int(df.loc[move_damage_index[0]+1].value.split('has ')[1].split(' hp')[0])
    currenthp_dict[current_battlers[defender]] = current_hp
    damage_string = "|-damage|p" + str(defender) + "a: " +  current_battlers[defender] + "|" + str(current_hp) + r"\/" + str(maxhp_dict[current_battlers[defender]])
    battle_render.append(damage_string)
    return

In [4]:
seq_list = []
timestamp_list = []
category_list = []
value_list = []
for ln in range(len(battle_log)):

    s  = battle_log[ln]
    s1 = s[s.find("(")+1:s.find(")")].split(', ')
    for i in s1:
        i = i.split('=')
        if i[0] == 'seq':
            seq_list.append(i[1])
        if i[0] == 'timestamp':
            timestamp_list.append(i[1])
        if i[0] == 'category':
            category_list.append(i[1])
        if i[0] == 'value':
            value_list.append('='.join(i[1:]))        

In [5]:
log_pd = pd.DataFrame(zip(seq_list,timestamp_list,category_list,value_list))
log_pd.columns = ['seq', 'timestamp', 'category', 'value']
log_pd.category = log_pd.category.astype('str')
log_pd.value = log_pd.value.astype('str')

log_pd_l = [y for x, y in log_pd.groupby('timestamp', as_index=False)]

In [6]:
battle_render = []
maxhp_dict = {}
currenthp_dict = {}
current_battlers = {}

header = ''

df = log_pd_l[0]
party_members = df[df.category == "'join_party'"]
for line in party_members.value:
    poke_name = line.partition("PokemonId.")[2].partition(":")[0].capitalize()
    if poke_name.find('alolan') != -1:
        poke_name = poke_name.split('_')[0]+"-Alola"
        return(poke_name.capitalize())

    poke_health = round(float(line.partition("health=")[2].partition(" ")[0]))
    maxhp_dict[poke_name] = poke_health
    currenthp_dict[poke_name] = poke_health

t1a = df[df.category == "'team1_active'"].value.iloc[0]
poke_name = t1a.partition("PokemonId.")[2].partition(" ")[0].capitalize()
if poke_name.find('alolan') != -1:
    poke_name = poke_name.split('_')[0]+"-Alola"
battle_render.append("|switch|p1a: " + poke_name + "|" + poke_name + ", M|"+str(currenthp_dict[poke_name])+r"\/"+str(maxhp_dict[poke_name]))
current_battlers[1] = poke_name


t2a = df[df.category == "'team2_active'"].value.iloc[0]
poke_name = t2a.partition("PokemonId.")[2].partition(" ")[0].capitalize()
if poke_name.find('alolan') != -1:
    poke_name = poke_name.split('_')[0]+"-Alola"
battle_render.append("|switch|p2a: " + poke_name + "|" + poke_name + ", M|"+str(currenthp_dict[poke_name])+r"\/"+str(maxhp_dict[poke_name]))
current_battlers[2] = poke_name

battle_render.append("|turn|1")
battle_render.append("|")
battle_switch = 1


In [7]:
df = log_pd_l[2]

In [8]:
attack_count = list(df.index[df.category.isin(["'Attack'", "'Fast Attack'"])])
if len(attack_count) > 1: 
    attack_1_df = df.loc[:attack_count[1]-1]
    attack_2_df = df.loc[attack_count[1]:]
    process_attack(attack_1_df)
    process_attack(attack_2_df)
elif len(attack_count) == 1: 
    process_attack(df)

In [9]:
battle_render

['|switch|p1a: Magikarp|Magikarp, M|10\\/10',
 '|switch|p2a: Rattata|Rattata, M|22\\/22',
 '|turn|1',
 '|',
 '|-prepare|p2a: Rattata|Geomancy',
 '|-singleturn|p1a: Magikarp|Protect',
 '|move|p2a: Rattata|Body Slam|p1a: Magikarp',
 '|-damage|p1a: Magikarp|6\\/10',
 '|move|p2a: Rattata|Tackle|p1a: Magikarp',
 '|-damage|p1a: Magikarp|4\\/10']

In [ ]:
i = 5

In [17]:
i = 5

In [26]:
i += 1
log_pd_l[i]


,seq,timestamp,category,value
55,56,2000.0,'Fast Attack','team 1 magikarp used SPLASH on team 2 rattata'
56,57,2000.0,'Attack Charge Up','team 1 magikarp now has 12 energy'
57,58,2000.0,'Fast Attack Damage','team 2 rattata took 1 damage: '
58,59,2000.0,'Health','team 2 rattata has 21 hp left'
59,60,2000.0,'Fast Attack','team 1 magikarp used SPLASH on team 2 rattata'
60,61,2000.0,'Attack Charge Up','team 1 magikarp now has 24 energy'
61,62,2000.0,'Fast Attack Damage','team 2 rattata took 1 damage: '
62,63,2000.0,'Health','team 2 rattata has 21 hp left'
63,64,2000.0,'Team1 Faint','team 2 rattata KOs team 1 magikarp'
64,65,2000.0,'FocusBand post_combat','1 magikarp revived with 6.167810037732113 hp ...


In [31]:
i += 1
log_pd_l[i]


,seq,timestamp,category,value
72,73,2500.0,'Attack','team 1 magikarp used STRUGGLE on team 2 rattata'
73,74,2500.0,'Energy Use','team 1 magikarp has 0 energy left'
74,75,2500.0,'Charged Move Damage','team 2 rattata took 15 damage: '
75,76,2500.0,'Health','team 2 rattata has 6 hp left'
76,77,2500.0,'Team1 Faint','team 2 rattata KOs team 1 magikarp'
77,78,2500.0,'FocusBand post_combat','1 magikarp revived with 6.167810037732113 hp ...
78,79,2500.0,'Turn','26'
79,80,2500.0,'team1_active','PokemonId.magikarp is fighting for team 1'
80,81,2500.0,'team2_active','PokemonId.rattata is fighting for team2'


In [44]:
i += 1
log_pd_l[i]


,seq,timestamp,category,value
112,113,3700.0,'Turn','38'
